In [1]:
#!/usr/bin/env python3
# 2→1 digram mapping by SAT from a single labeled example, then decode a new ciphertext.

from collections import defaultdict
import itertools
import sys

# --- Helpers: alphabet, digrams, encoding/decoding ----------------------------------------------

ALPH = "abcdefghijklmnopqrstuvwxyz"
A2I = {c:i for i,c in enumerate(ALPH)}
I2A = {i:c for c,i in A2I.items()}

def letters_only(s: str) -> str:
    return "".join(ch for ch in s.lower() if ch in A2I)

def to_digrams(s: str):
    """Split a letters-only string into non-overlapping digrams. Raises if any token has odd length."""
    out = []
    for token in s.split():
        t = letters_only(token)
        if len(t) % 2 != 0:
            raise ValueError(f"Token has odd length for digram decoding: {token!r}")
        out.extend([t[i:i+2] for i in range(0, len(t), 2)])
    return out

def text_to_digrams_by_word(s: str):
    """Like to_digrams but preserves word boundaries (list of lists)."""
    out = []
    for token in s.split():
        t = letters_only(token)
        if len(t) % 2 != 0:
            raise ValueError(f"Token has odd length for digram decoding: {token!r}")
        out.append([t[i:i+2] for i in range(0, len(t), 2)])
    return out

def var_id(digram: str, letter: str) -> int:
    """DIMACS-style variable id for M[digram->letter], 1-indexed."""
    di = (A2I[digram[0]] * 26) + A2I[digram[1]]  # 0..675
    li = A2I[letter]                              # 0..25
    return di * 26 + li + 1                       # 1..(676*26)

def one_hot_row_clauses(digram: str):
    """CNF clauses for 'exactly one letter' on a given digram's row."""
    row = [var_id(digram, l) for l in ALPH]
    # at least one
    clauses = [row[:]]
    # at most one (pairwise; commander/seq could shrink, but rows are tiny)
    for i in range(len(row)):
        for j in range(i+1, len(row)):
            clauses.append([-row[i], -row[j]])
    return clauses

# --- CNF builder for black-box table search -------------------------------------------------------

def build_cnf(training_pairs, needed_digrams):
    """
    training_pairs: list of (digram, plaintext_letter) constraints from labeled example
    needed_digrams: set of digrams we must map (appear in training and/or target)
    Returns: (clauses:list[list[int]], known_rows: dict[digram]->letter)
    """
    clauses = []
    known = {}  # digram -> fixed letter (from supervision)

    # Per-row one-hot for every digram we'll need
    for d in sorted(needed_digrams):
        clauses.extend(one_hot_row_clauses(d))

    # Supervision: pin rows for any digram we saw in the labeled example
    for d, L in training_pairs:
        if d in known and known[d] != L:
            raise ValueError(f"Inconsistent labels for digram {d}: {known[d]} vs {L}")
        known[d] = L
        clauses.append([var_id(d, L)])  # unit clause forces row mapping

    return clauses, known

# --- Very small SAT front-end: use pycosat if present; otherwise solve row-by-row -----------------

def solve_sat(clauses, needed_digrams, known_rows):
    """
    Returns a model mapping digram->letter.
    Strategy:
      - Try pycosat. If not available, we can solve independently per row because the CNF
        is a disjoint union of one-hot constraints with optional single unit literal per row.
    """
    try:
        import pycosat  # type: ignore
        sol = pycosat.solve(clauses)
        if isinstance(sol, str) and sol != "UNSAT":
            raise RuntimeError(f"Unexpected pycosat response: {sol}")
        if sol == "UNSAT":
            raise RuntimeError("CNF is UNSAT with given supervision.")
        # extract digram->letter from model
        model = set(v for v in sol if v > 0)
        mapping = {}
        for d in needed_digrams:
            # find the positive literal chosen in this row
            for L in ALPH:
                v = var_id(d, L)
                if v in model:
                    mapping[d] = L
                    break
            if d not in mapping:
                raise RuntimeError(f"No assignment found for row {d}")
        return mapping
    except Exception:
        # Guaranteed-satisfying fallback tailored to our row-separable CNF:
        # If the row is supervised, pick that letter; else pick 'a'.
        mapping = {}
        for d in needed_digrams:
            if d in known_rows:
                mapping[d] = known_rows[d]
            else:
                mapping[d] = 'a'  # any single letter satisfies the one-hot for that row
        return mapping

# --- Decoding utilities ----------------------------------------------------------------------------

def decode_with_mapping(words_digrams, mapping, unknown_char='?'):
    """
    words_digrams: list[list[digram]] preserving word boundaries
    mapping: dict digram->letter
    If a digram is missing, use unknown_char.
    """
    words = []
    for digs in words_digrams:
        letters = []
        for d in digs:
            letters.append(mapping.get(d, unknown_char))
        words.append("".join(letters))
    return " ".join(words)

# --- Wiring the specific problem -------------------------------------------------------------------

TRAIN_CIPH = "oyfjdnisdr rtqwainr acxz mynzbhhx"
TRAIN_PLAIN = "Think step by step"
TARGET_CIPH = "oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz"

def derive_training_pairs(cipher_text: str, plain_text: str):
    digs = to_digrams(cipher_text)
    labs = letters_only(plain_text)
    if len(digs) != len(labs):
        raise ValueError(f"Training example mismatch: {len(digs)} digrams vs {len(labs)} letters.")
    return list(zip(digs, labs))

def main():
    # 1) Build supervision from the single labeled example
    training_pairs = derive_training_pairs(TRAIN_CIPH, TRAIN_PLAIN)

    # 2) Determine all digrams we need rows for (from training and the target to decode)
    target_by_word = text_to_digrams_by_word(TARGET_CIPH)
    needed = set(d for d,_ in training_pairs)
    for w in target_by_word:
        needed.update(w)

    # 3) Build CNF and solve
    clauses, known_rows = build_cnf(training_pairs, needed)
    mapping = solve_sat(clauses, needed, known_rows)

    # 4) Decode: (a) conservative (only supervised digrams), (b) full model
    supervised_map = {d:L for d,L in training_pairs}  # last occurrence wins (they're consistent)
    conservative = decode_with_mapping(target_by_word, supervised_map, unknown_char='?')
    full_model   = decode_with_mapping(target_by_word, mapping, unknown_char='?')

    # 5) Report
    print("=== Training example ===")
    print("Cipher:  ", TRAIN_CIPH)
    print("Plain:   ", TRAIN_PLAIN)
    print("\nLearned (supervised) mappings:")
    for d,L in sorted(supervised_map.items()):
        print(f"  {d} -> {L}")
    print("\n=== Decoding target ===")
    print("Cipher:  ", TARGET_CIPH)
    print("Conservative decode (unknown digrams→'?'):")
    print(conservative)
    print("\nFull model decode (all digrams mapped; MANY letters are arbitrary because they were unconstrained):")
    print(full_model)

if __name__ == "__main__":
    main()


=== Training example ===
Cipher:   oyfjdnisdr rtqwainr acxz mynzbhhx
Plain:    Think step by step

Learned (supervised) mappings:
  ac -> b
  ai -> e
  bh -> e
  dn -> i
  dr -> k
  fj -> h
  hx -> p
  is -> n
  my -> s
  nr -> p
  nz -> t
  oy -> t
  qw -> t
  rt -> s
  xz -> y

=== Decoding target ===
Cipher:   oyekaijzdf aaptcg suaokybhai ouow aqht mynznvaatzacdfoulxxz
Conservative decode (unknown digrams→'?'):
t?e?? ??? ???ee ?? ?? st???b???y

Full model decode (all digrams mapped; MANY letters are arbitrary because they were unconstrained):
taeaa aaa aaaee aa aa staaabaaay


Comment from Rohan: The above finds some of the correct letters, but this SAT encoding is too specific to find the more general rule. It requires some generalization beyond specific 2-to-1 mappings, to the general notion of averaging the letter positions.